In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.0 MB/s eta 0:00:00


In [37]:
from PIL import Image
from transformers import pipeline

vqa_pipeline = pipeline("visual-question-answering")

image =  Image.open("/content/images.jpeg")
question = "what man is doing in image?"
ques2 = 'Is there any animal in image?'
ques3 = 'How many plants are there in image?'
vqa_pipeline(image, question, top_k=1)
vqa_pipeline(image, ques2, top_k=1)
vqa_pipeline(image, ques3, top_k=1)

No model was supplied, defaulted to dandelin/vilt-b32-finetuned-vqa and revision 4355f59 (https://huggingface.co/dandelin/vilt-b32-finetuned-vqa).
Using a pipeline without specifying a model name and revision in production is not recommended.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


[{'score': 0.4866306781768799, 'answer': '3'}]

In [6]:
import torch
from transformers import BertTokenizer, TFBertModel

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertModel.from_pretrained('bert-base-cased')
ques = "What is man doing in image?"

def ques_embed(ques):

  encoded_ques = tokenizer(ques,return_tensors='tf')
  output = model(encoded_ques)
  #print(output)
  return output

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [28]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np

model = ResNet50(include_top=False,pooling='avg')

def image_embed(image_path):
  img = image.load_img(image_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  x = preprocess_input(x)
  pred = model.predict(x)
  print(pred)
  return pred

In [22]:
image_embed('/content/images.jpeg')

1/1 [==============================] - 1s 1s/step
[[0.3645531  0.55496246 0.7950819  ... 1.0758163  0.8979599  0.30728483]]


array([[0.3645531 , 0.55496246, 0.7950819 , ..., 1.0758163 , 0.8979599 ,
        0.30728483]], dtype=float32)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

ques_embed_dim = 768
image_embed_dim = 2048
common_embed_dim = 512

ques_projection = nn.Linear(ques_embed_dim,common_embed_dim)
image_projection = nn.Linear(image_embed_dim,common_embed_dim)


a = ques_embed(ques)

# Project the embeddings
question_embedding_projected = ques_projection(a)
image_embedding_projected = image_projection(image_embed('/content/images.jpeg'))

combined_embedding = question_embedding_projected * image_embedding_projected


In [26]:
def VQA(common_embed_dim,num_classes):
  fc = nn.Sequential(nn.Linear(common_embed_dim,256),nn.ReLU(),nn.Linear(256,num_classes))
  def forward(combined_embedding):
    output = fc(combined_embedding)
    return F.log_softmax(output,dim=1)

vqa = VQA(common_embed_dim,1)




In [ ]:
#Define loss function and optimizer
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(vqa.parameters(),lr=0.001)

In [ ]:
output = vqa_model(combined_embedding)


In [ ]:
# Calculate loss and backpropagate
target = torch.randint(0, 1, (1,))  # Replace with your actual ground truth label
loss = criterion(output, target)
loss.backward()

In [ ]:
optimizer.step()